# attention의 구조

In [2]:
import tensorflow as tf
from tensorflow.keras import Model, layers

In [10]:
# 인코더 - eng
enc_input = layers.Input(shape=(4, 171))   # 입력하는 영어 단어의 길이는 4
enc_output, state_h, state_c = layers.LSTM(128, return_sequences=True, return_state=True)(enc_input)

# 디코더 - kor
dec_input = layers.Input(shape=(3, 171))   # 1-start토큰 + 문자2개
dec_output, _, _ = layers.LSTM(128, return_sequences=True, return_state=True)(dec_input, initial_state=[state_h, state_c])

In [12]:
# 어텐션 매커니즘
context_vector = layers.Attention()([dec_output, enc_output])
context_and_lstm_output = layers.Concatenate()([context_vector, dec_output])

In [13]:
# 디코더의 출력층 정의
output = layers.Dense(171, activation='softmax')(context_and_lstm_output)

In [15]:
# 모델 생성
model = Model(inputs=[enc_input, dec_input], outputs=[output])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 4, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 3, 171)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_8 (LSTM)       │ [(None, 4, 128),  │    153,600 │ input_layer_8[0]… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_9 (LSTM)       │ [(None, 3, 128),  │    153,600 │ input_layer_9[0]… │
│                     │ (None, 128),      │            │ lstm_8[0][1],     │
│                     │ (None, 128)]      │            │ lstm_8[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 3, 128)    │          0 │ lstm_9[0][0],     │
│ (Attention)         │                   │            │ lstm_8[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3, 256)    │          0 │ attention_1[0][0… │
│ (Concatenate)       │                   │            │ lstm_9[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 171)    │     43,947 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 351,147 (1.34 MB)

 Trainable params: 351,147 (1.34 MB)

 Non-trainable params: 0 (0.00 B)

# eng -> kor 번역

In [16]:
# 손실함수와 옵티마이저 지정
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [20]:
import pandas as pd
import numpy as np

# 알파벳 문자열 생성
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']   # 전체 문자배열
arr2 = pd.read_csv('korean.csv', header=None)

# 알파벳과 한글 매핑
num_to_char = arr1 + arr2[0].values.tolist()
len(num_to_char)

171

# 학습용 데이터셋 설정

In [23]:
# num_to_char을 숫자, 딕셔너리로 만들기
char_to_num = {char:i for i, char in enumerate(num_to_char)}
char_to_num

{'S': 0,
 'E': 1,
 'P': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '가': 29,
 '각': 30,
 '간': 31,
 '감': 32,
 '개': 33,
 '거': 34,
 '것': 35,
 '게': 36,
 '계': 37,
 '고': 38,
 '관': 39,
 '광': 40,
 '구': 41,
 '굴': 42,
 '규': 43,
 '그': 44,
 '금': 45,
 '기': 46,
 '깊': 47,
 '나': 48,
 '날': 49,
 '남': 50,
 '내': 51,
 '넓': 52,
 '녀': 53,
 '노': 54,
 '놀': 55,
 '농': 56,
 '높': 57,
 '뉴': 58,
 '늦': 59,
 '다': 60,
 '단': 61,
 '도': 62,
 '동': 63,
 '들': 64,
 '람': 65,
 '랑': 66,
 '래': 67,
 '램': 68,
 '류': 69,
 '름': 70,
 '릎': 71,
 '리': 72,
 '많': 73,
 '망': 74,
 '매': 75,
 '머': 76,
 '먼': 77,
 '멍': 78,
 '메': 79,
 '명': 80,
 '모': 81,
 '목': 82,
 '무': 83,
 '물': 84,
 '미': 85,
 '바': 86,
 '반': 87,
 '방': 88,
 '번': 89,
 '복': 90,
 '부': 91,
 '분': 92,
 '붕': 93,
 '비': 94,
 '뿌': 95,
 '사': 96,
 '상': 97,
 '색': 98,
 '생': 99,
 '서': 100,

In [25]:
raw = pd.read_csv('translate.csv', header=None)
raw.values.tolist()

[['cold', '감기'],
 ['come', '오다'],
 ['cook', '요리'],
 ['copy', '복사'],
 ['cost', '비용'],
 ['date', '날짜'],
 ['deal', '거래'],
 ['deep', '깊은'],
 ['desk', '책상'],
 ['down', '아래'],
 ['dust', '먼지'],
 ['duty', '의무'],
 ['each', '각각'],
 ['east', '동쪽'],
 ['easy', '쉽다'],
 ['exit', '탈출'],
 ['face', '얼굴'],
 ['fact', '사실'],
 ['fall', '가을'],
 ['farm', '농장'],
 ['feet', '다리'],
 ['find', '찾다'],
 ['fine', '좋다'],
 ['fish', '생선'],
 ['flow', '흐름'],
 ['fund', '기금'],
 ['gain', '수익'],
 ['game', '놀이'],
 ['gift', '선물'],
 ['girl', '소녀'],
 ['give', '주다'],
 ['goal', '목적'],
 ['gray', '회색'],
 ['hair', '머리'],
 ['harm', '피해'],
 ['hell', '지옥'],
 ['help', '도움'],
 ['high', '높은'],
 ['hole', '구멍'],
 ['hope', '소망'],
 ['hour', '시각'],
 ['join', '합류'],
 ['kiss', '키스'],
 ['knee', '무릎'],
 ['lady', '부인'],
 ['late', '늦은'],
 ['left', '왼쪽'],
 ['life', '생명'],
 ['loss', '손해'],
 ['love', '사랑'],
 ['luck', '행운'],
 ['mail', '우편'],
 ['male', '남자'],
 ['many', '많은'],
 ['meal', '식사'],
 ['meat', '고기'],
 ['menu', '메뉴'],
 ['milk', '우유'],
 ['name', '이름'

In [26]:
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [27]:
eng_kor[0]

['cold', '감기']

# 단어 -> 숫자로 변환

In [29]:
temp_eng = 'love'   # 임시 입력 단어
temp_eng_n = [char_to_num[c] for c in temp_eng]
np.eye(171)[temp_eng_n]   # 대각행렬

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.,

# 학습을 위한 인코딩 함수

In [30]:
# 단어를 원-핫 인코딩된 배열로 변환하는 함수
def encode(eng_kor):#번역데이터
    enc_in = []   # 인코더 입력 (영어 단어 → 원-핫 벡터)
    dec_in = []   # 디코더 입력 (한국어 단어 앞에 시작 토큰 'S' 추가)
    rnn_out = []  # 디코더 출력 (한국어 단어 뒤에 종료 토큰 'E' 추가)

    for seq in eng_kor:
        # 1. 영어 단어를 숫자 인덱스로 변환 후 원-핫 인코딩
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng])   # 171차원 one-hot 벡터

        # 2. 한국어 단어 앞에 'S' 추가 후 인덱스로 변환 → 디코더 입력
        kor = [char_to_num[c] for c in ('S' + seq[1])]
        dec_in.append(np.eye(171)[kor])   # 디코더 입력도 원-핫 인코딩

        # 3. 한국어 단어 뒤에 'E' 추가 후 인덱스로 변환 → 디코더 출력(정답)
        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    # 리스트를 넘파이 배열로 변환
    enc_in = np.array(enc_in)      # (샘플 수, 단어 길이, 171)
    dec_in = np.array(dec_in)      # (샘플 수, 단어 길이+1, 171)
    rnn_out = np.array(rnn_out)    # (샘플 수, 단어 길이+1)

    # 디코더 출력에 차원 추가 → (샘플 수, 단어 길이+1, 1)
    rnn_out = np.expand_dims(rnn_out, axis=2)

    return enc_in, dec_in, rnn_out

In [31]:
sample = [['word', '단어']]
encode(sample)

(array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [33]:
X_enc, X_dec, y_rnn = encode(eng_kor)
# 학습
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1783
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1720
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1684
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1635
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1571
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1508
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1476
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1435
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1384
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1340
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1316
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1287
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1239
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1199
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1166
Epoch 16/500
4/4 ━━

# 테스트

In [ ]:
model.predict(['tall', 'PP'])   # 임의의 값을 넣어서 테스트해보기
# error

ValueError: Unrecognized data type: x=['tall', 'PP'] (of type <class 'list'>)

In [36]:
enc_in, dec_in, _ = encode([['tall', 'PP']])   # 받아서

In [40]:
pred = model.predict([enc_in, dec_in])   # 예측
word = np.argmax(pred[0], axis=-1)   # 예측값들 중 가장 큰 값
num_to_char[word[0]], num_to_char[word[1]], num_to_char[word[2]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


('크', '다', 'E')

# 여러 단어 [ ] -> 출력하는 프로그램 완성

In [47]:
words = [['paln', 'life', 'free', 'tall']]
word_add = []

for i in words :
    enc_in, dec_in, _ = encode(words)
    pred = model.predict([enc_in, dec_in])
    word = np.argmax(pred[0], axis=-1)
    # 글자를 합치는 단계
    word_add.append(word)

    num_to_char[word[0]], num_to_char[word[1]], num_to_char[word[2]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
